In [1]:
# https://stackoverflow.com/questions/21321670/how-to-change-fonts-in-matplotlib-python
# https://towardsdatascience.com/ways-to-evaluate-regression-models-77a3ff45ba70
# https://www.kaggle.com/pmarcelino/comprehensive-data-exploration-with-python

import pandas as pd
import numpy  as np
import json
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)
warnings.simplefilter("ignore")
hfont = {'fontname':'Monaco'}
hfont = {'fontsize':'24'}

# Making the plots standard 
%matplotlib inline
plt.rcParams["figure.figsize"] = [16, 5]

In [2]:
#import the training and test pickles 
train_df = pd.read_pickle('train_df.pickle')
test_df = pd.read_pickle('test_df.pickle')

#get the training std 
trainrtt_mean = train_df['last_rtt'].mean()
trainrtt_std = train_df['last_rtt'].std()

In [3]:
print(trainrtt_mean,trainrtt_std)

16.864126474595782 9.342051656026516


In [4]:
# Choose your features and target
X_train = train_df['normalizzed_distance'].values.reshape(-1,1)
y_train = train_df['normalizzed_rtt'].values
        
X_test = test_df['normalizzed_distance'].values.reshape(-1,1)
y_test = test_df['normalizzed_rtt'].values



In [6]:
#Importing the module
import pmdarima as pm

#first model 
arima_model = pm.auto_arima(y_train,test="adf",trace=True,error_action='ignore',supress_warnings = True)

#Summary of the model
arima_model.summary()

Performing stepwise search to minimize aic
 ARIMA(2,0,2)(0,0,0)[0] intercept   : AIC=-1512583.286, Time=29.70 sec
 ARIMA(0,0,0)(0,0,0)[0] intercept   : AIC=2207017.157, Time=4.77 sec
 ARIMA(1,0,0)(0,0,0)[0] intercept   : AIC=inf, Time=14.84 sec
 ARIMA(0,0,1)(0,0,0)[0] intercept   : AIC=1217492.557, Time=67.40 sec
 ARIMA(0,0,0)(0,0,0)[0]             : AIC=2207015.157, Time=2.45 sec
 ARIMA(1,0,2)(0,0,0)[0] intercept   : AIC=-1512586.126, Time=25.75 sec
 ARIMA(0,0,2)(0,0,0)[0] intercept   : AIC=496875.858, Time=198.87 sec
 ARIMA(1,0,1)(0,0,0)[0] intercept   : AIC=-1512585.764, Time=19.94 sec
 ARIMA(1,0,3)(0,0,0)[0] intercept   : AIC=-1512587.456, Time=79.52 sec
 ARIMA(0,0,3)(0,0,0)[0] intercept   : AIC=27563.201, Time=218.16 sec
 ARIMA(2,0,3)(0,0,0)[0] intercept   : AIC=-1512585.006, Time=62.45 sec
 ARIMA(1,0,4)(0,0,0)[0] intercept   : AIC=-1512585.046, Time=105.40 sec
 ARIMA(0,0,4)(0,0,0)[0] intercept   : AIC=-325387.093, Time=261.10 sec
 ARIMA(2,0,4)(0,0,0)[0] intercept   : AIC=-1513794

<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:                      y   No. Observations:               777699
Model:               SARIMAX(5, 0, 5)   Log Likelihood              774772.607
Date:                Mon, 29 Jan 2024   AIC                       -1549523.214
Time:                        15:22:09   BIC                       -1549396.009
Sample:                             0   HQIC                      -1549487.851
                             - 777699                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1          0.7966      0.005    158.225      0.000       0.787       0.806
ar.L2          0.1838      0.005     40.038      0.000       0.175       0.193
ar.L3          0.6200      0.002    310.790      0.000       0.616       0.624
ar.L4         -0.4528      0.006    -81.275      0.000      -0.464      -0.442
ar.L5         -0.1482      0.005    -30.436      0.000      -0.158      -0.139
ma.L1          0.1515      0.005     30.917      0.000       0.142       0.161
ma.L2         -0.0495      0.002    -28.434      0.000      -0.053      -0.046
ma.L3         -0.6885      0.001   -604.599      0.000      -0.691      -0.686
ma.L4         -0.2170      0.005    -41.638      0.000      -0.227      -0.207
ma.L5         -0.1054      0.001   -204.794      0.000      -0.106      -0.104
sigma2         0.0080   6.81e-07   1.17e+04      0.000       0.008       0.008
===================================================================================
Ljung-Box (L1) (Q):                  56.78   Jarque-Bera (JB):       39975454603.82
Prob(Q):                              0.00   Prob(JB):                         0.00
Heteroskedasticity (H):               1.03   Skew:                             3.83
Prob(H) (two-sided):                  0.00   Kurtosis:                      1113.67
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [7]:
import statsmodels.api as sm

# Get the best order
order = arima_model.order
model = sm.tsa.ARIMA(y_test, order=order)
arima_model = model.fit()

# Make predictions on the test set
arima_predictions = arima_model.fittedvalues

/opt/homebrew/lib/python3.10/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [8]:
# calculate the mean squared error and standard deviation
from sklearn.metrics import mean_squared_error
rmse_arima =  np.sqrt(mean_squared_error(arima_predictions,y_test))
drmse_arima = rmse_arima * trainrtt_std
print(f"Root Mean Squared Error and denormalised rmse: {rmse_arima,drmse_arima}")

Root Mean Squared Error and denormalised rmse: (0.08935129185566222, 0.8347243840482979)
